In [12]:
import torch 
import torch.nn.functional as F
from torch import nn
from torch import optim

In [17]:
from torchvision import datasets, transforms
transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])

trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download = True, train = True, transform = transforms)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle = True)
testset =  datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download = True, train = False, transform = transforms)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle = True)

In [21]:
class Classifier(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
        self.dropout = nn.Dropout(p=0.2)
    
    def forward(self, x):
        #flatten input tensor
        x = x.view(x.shape[0], -1)
        
        #apply dropout
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        
        #apply log_softmax to output and no dropout here
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x     

In [23]:
model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.003)

epochs = 30
steps = 0

train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        optimizer.zero_grad()
        
        log_ps = model.forward(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    else:
        test_loss = 0
        accuracy = 0
        
        with torch.no_grad():
            for images, labels in trainloader:
                log_ps = model.forward(images)
                test_loss += criterion(log_ps, labels)
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim = 1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
                
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(trainloader))

        print ('Epoch : {}/{}..'.format(e+1, epochs),
               'Training Loss : {: .3f}..'.format(running_loss/len(trainloader)),
               'Testing Accuracy : {: .3f}..'.format(accuracy/len(trainloader)))

Epoch : 1/30.. Training Loss :  0.602.. Testing Accuracy :  0.835..
Epoch : 2/30.. Training Loss :  0.482.. Testing Accuracy :  0.835..
Epoch : 3/30.. Training Loss :  0.449.. Testing Accuracy :  0.852..
Epoch : 4/30.. Training Loss :  0.433.. Testing Accuracy :  0.866..
Epoch : 5/30.. Training Loss :  0.418.. Testing Accuracy :  0.852..
Epoch : 6/30.. Training Loss :  0.410.. Testing Accuracy :  0.866..
Epoch : 7/30.. Training Loss :  0.406.. Testing Accuracy :  0.842..
Epoch : 8/30.. Training Loss :  0.405.. Testing Accuracy :  0.859..
Epoch : 9/30.. Training Loss :  0.392.. Testing Accuracy :  0.859..
Epoch : 10/30.. Training Loss :  0.396.. Testing Accuracy :  0.867..
Epoch : 11/30.. Training Loss :  0.386.. Testing Accuracy :  0.864..
Epoch : 12/30.. Training Loss :  0.387.. Testing Accuracy :  0.868..
Epoch : 13/30.. Training Loss :  0.376.. Testing Accuracy :  0.868..
Epoch : 14/30.. Training Loss :  0.378.. Testing Accuracy :  0.871..
Epoch : 15/30.. Training Loss :  0.375.. Te